In [ ]:
!pip install wandb -q
!wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.9 MB/s eta 0:00:00
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import pandas as pd
import wandb

In [ ]:
def prepare_data(file_name_dataset):
    data = pd.read_csv(file_name_dataset)

    tv = data['TV'].values
    radio = data['Radio'].values
    newspaper = data['Newspaper'].values
    sales = data['Sales'].values

    X = [tv,radio,newspaper]
    y = sales
    return X,y

In [ ]:
# Predict
def predict(x1, x2, x3, w1, w2, w3, b):
    result = x1*w1 + x2*w2 + x3*w3 + b
    return result

# Loss
def compute_loss(y_hat,y):
    loss = (y_hat - y)*(y_hat - y) / 2
    return loss

# Gradient
def compute_gradient_wi(xi, y, y_hat):
    dwi = 2*xi*(y_hat-y)
    return dwi
def compute_gradient_b(y, y_hat):
    db = 2*(y_hat-y)
    return db

# Update weight
def update_weight_wi(wi,dwi,lr):
    wi_new = wi - lr*dwi
    return wi_new
def update_weight_b(b,db,lr):
    b_new = b - lr*db
    return b_new

In [ ]:
X, y = prepare_data('/content/drive/MyDrive/Colab Notebooks/40-Day Python/Day-38_data/advertising.csv')

dataset = pd.DataFrame({
    'tv': X[0],
    'radio': X[1],
    'newspaper': X[2],
    'sales': y})

# init weights
lr = 0.01
epochs = 1000
w1 = 0
w2 = 0
w3 = 0
b = 1

# init project wandb
wandb.init(
    # Set the project where this run will be logged
    project = 'kn-linear-regression',
    config={
        'learning_rate': lr,
        'epochs': epochs
    }
)

wandb.run.log({'Dataset': wandb.Table(dataframe=dataset)})

X_train = [dataset['tv'],dataset['radio'],dataset['newspaper']]
Y_train = dataset['sales']

N = len(Y_train)

# parameter
losses = [] # for debug

for epoch in range(epochs):
    for i in range(N):
        x1 = X_train[0][i]
        x2 = X_train[1][i]
        x3 = X_train[2][i]

        y = Y_train[i]

        # predict y_hat
        y_hat = predict(x1,x2,x3, w1,w2,w3, b)

        # compute loss
        loss = compute_loss(y_hat,y)
        # tracking loss with WandB
        wandb.log({'loss': loss})


        # compute gradient
        dw1 = compute_gradient_wi(x1, y, y_hat)
        dw2 = compute_gradient_wi(x2, y, y_hat)
        dw3 = compute_gradient_wi(x3, y, y_hat)
        db = compute_gradient_b(y, y_hat)

        # update weight
        w1 = update_weight_wi(w1,dw1,lr)
        w2 = update_weight_wi(w2,dw2,lr)
        w3 = update_weight_wi(w3,dw3,lr)
        b = update_weight_b(b, db, lr)

# Mark a run as finished, and finish uploading all data.
wandb.finish()

wandb: Currently logged in as: noravnhuynh (norahuynh). Use `wandb login --relogin` to force relogin


<ipython-input-4-80efcba3a437>:8: RuntimeWarning: overflow encountered in scalar multiply
  loss = (y_hat - y)*(y_hat - y) / 2
<ipython-input-4-80efcba3a437>:13: RuntimeWarning: overflow encountered in scalar multiply
  dwi = 2*xi*(y_hat-y)
<ipython-input-4-80efcba3a437>:21: RuntimeWarning: invalid value encountered in scalar subtract
  wi_new = wi - lr*dwi


loss,nan
